<a href="https://colab.research.google.com/github/DeependraParichha1004/DeepLearningProjects/blob/main/AiCaption_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.9 MB/s eta 0:00:00


In [2]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.2/294.2 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 5.2 MB/s eta 0:00:00
   

In [3]:
# import torch
import transformers
# import gradio
import re
from transformers import AutoTokenizer, ViTFeatureExtractor, VisionEncoderDecoderModel
import PIL
from PIL import Image


In [4]:
device='cpu'
encoder_checkpoint = "nlpconnect/vit-gpt2-image-captioning"
decoder_checkpoint = "nlpconnect/vit-gpt2-image-captioning"
model_checkpoint = "nlpconnect/vit-gpt2-image-captioning"
feature_extractor = ViTFeatureExtractor.from_pretrained(encoder_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(decoder_checkpoint)
model = VisionEncoderDecoderModel.from_pretrained(model_checkpoint).to(device)

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [5]:
# !pip install albumentations
import albumentations as A
import cv2
import matplotlib.pyplot as plt
def visualize(image):
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(image)

In [6]:
import cv2
import albumentations as A
image = cv2.imread('/content/MynewImage.jpeg')
print(image.shape)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

transform = A.Compose(
    [A.CLAHE(),
    #  A.RandomRotate90(),
     A.Transpose(),
     A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50,
                        rotate_limit=45, p=.75),
    #  A.Blur(blur_limit=3),
    #  A.OpticalDistortion(),
    #  A.GridDistortion(),
     A.HueSaturationValue()])

augmented_image = transform(image=image)['image']
# visualize(augmented_image)

(1600, 1200, 3)


In [7]:
def image_augmentation2(ima,n_caps):
  imgs=[]
  for i in range(n_caps):
    # path='/content/download (2).png'
    # image = cv2.imread(path)
    # print(image.shape)
    imag = cv2.cvtColor(ima, cv2.COLOR_BGR2RGB)

    transform = A.Compose(
        [A.CLAHE(),
        #  A.RandomRotate90(),
        A.Transpose(),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50,
                            rotate_limit=45, p=.75),
        #  A.Blur(blur_limit=3),
        #  A.OpticalDistortion(),
        #  A.GridDistortion(),
        A.HueSaturationValue()])

    augmented_image = transform(image=imag)['image']
    # plt.savefig('/content/')
    # visualize(augmented_image)
    imgs.append(augmented_image)
  return imgs

In [8]:
def predict2(ima,n_caps):#,max_length=64, num_beams=4
  # image=Image.open(image_path)
  aug_img=image_augmentation2(ima,int(n_caps))
  caps=[]

  for img in aug_img:
    # print('caps',caps)
    # visualize(img)
    image = Image.fromarray(np.uint8(img)).convert('RGB')
    # image = image.convert('RGB')
    image = feature_extractor(image, return_tensors="pt").pixel_values.to(device)
    clean_text = lambda x: x.replace('<|endoftext|>','').split('\n')[0]
    caption_ids = model.generate(image, max_length = 64)[0]
    caption_text = clean_text(tokenizer.decode(caption_ids))
    caps.append(caption_text)
  #   caps+='\n'+caption_text
  # for i in range(1):
  #   print(caps,end=" ")
  return caps

In [9]:
import numpy as np#test
predict2(image,3)

['a man wearing a suit and tie ',
 'a man wearing a suit and tie ',
 'a row of black and white striped ties ']

In [ ]:
# def image_classifier(inp):
#   return type(inp)
demo2=gr.Interface(fn=predict2, inputs=["image","number"], outputs="text")
# demo = gr.Interface(fn=predict1, inputs=["image",gr.Dropdown(["one","multiple"])], outputs="text")
# demo = gr.Interface(fn=predict1, inputs=["image","number"], outputs="text")
demo2.launch(share=True)

In [10]:
import gradio as gr
with gr.Blocks(css=".gradio-container {background-color: grey}") as demo:
  img=gr.Image()
  a=gr.Number(label="No of captions")

  out=gr.Textbox(label="captions")
  btn=gr.Button("Click")
  def display(num,img):
    return int(num)
  btn.click(predict2,inputs=[img,a],outputs=out)
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://8db133661e449664bd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
with gr.Blocks(css=".gradio-container {background-color: grey}") as demo:
  img=gr.Image()
  a=gr.Number(label="No of captions")

  out=gr.Textbox()
  btn=gr.Button("Click")
  def display(num,img):
    return int(num)
  btn.click(predict2,inputs=[img,a],outputs=out)
demo.launch(share=True)

## Extra

In [ ]:
# PIL_image = Image.fromarray(np.uint8(numpy_image)).convert('RGB')

In [ ]:
# def predict(image_path,max_length=64, num_beams=4):
#   # image=Image.open(image_path)
#   aug_img=image_augmentation(image_path,1)
#   visualize(aug_img)
#   image = Image.fromarray(np.uint8(aug_img)).convert('RGB')
#   # image = image.convert('RGB')
#   image = feature_extractor(image, return_tensors="pt").pixel_values.to(device)
#   clean_text = lambda x: x.replace('<|endoftext|>','').split('\n')[0]
#   caption_ids = model.generate(image, max_length = max_length)[0]
#   caption_text = clean_text(tokenizer.decode(caption_ids))
#   return caption_text

In [ ]:
# import gradio as gr
# demo=gr.Interface(fn=predict1, inputs=["image","number"], outputs="text")
# # demo = gr.Interface(fn=predict1, inputs=["image",gr.Dropdown(["one","multiple"])], outputs="text")
# # demo = gr.Interface(fn=predict1, inputs=["image","number"], outputs="text")
# demo.launch()